In [ ]:
# Future improvements
# Separate dir for each step
# Create JSON array to assign IDs, 
#     keep track of PDF files process (each step?) etc.
# Implement 'fi'/'fl' check at beginning and end of words
# Remove numbers, urls

In [24]:
# Imports
import os # For file/directory interaction
import time, sys
import PyPDF2 # Library for converting PDF to text
from datetime import datetime, date # For log data
import decimal # Exceptions caught from this package
import re # For text replacement
import spacy # Pipeline processes (stopword and punctuation removal, lemmatization)
from nltk.stem.snowball import SnowballStemmer # Pipeline process for stemming
from nltk.corpus import words
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import json
nlp = spacy.load("en_core_web_sm")
txtFilesDir = 'Text Files'
rtnFilesDir = 'n Removed'
spaceFilesDir = 'No Spaces'
swFilesDir = 'Stop Words'
engFilesDir = 'English Words'
stemFilesDir = 'Stemmed'
jsonDocIndex = 'doc_dictionary.json'
jsonInvIndex = 'inverted_index.json'

absolute = 'C:/Users/micah/Documents/IWU/CIS Practicum/Files'

In [24]:
# Pre-condition: All PDF files to be processed are in the sub-directory
#     pdfDir, and pdfDir is in absPath. absPath is by default the 
#     directory in which the program is executed
# Post-condition: All PDF files processed without error are converted to
#     text files which are placed in a new sub-directory 'Text Files'
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter, process_pdf#process_pdf
from pdfminer.pdfparser import PDFParser
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams

from io import StringIO

def getText(pdfPath):
    rsrcmgr = PDFResourceManager()
    sio = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, sio, laparams=laparams)
#     interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    
    with open(pdfPath, 'rb') as pdfFile:
        process_pdf(rsrcmgr, device, pdfFile)
#         parser = PDFParser(pdfFile)
#         doc.set_parser(parser)
#         interpreter.process_pdf(doc)
#             layout = device.get_result()
#             for element in layout:
#                 if instanceof(element, LTTextBoxHorizontal):
#                     print(element.get_text)
                
    
    text = sio.getvalue()
    
    device.close()
    sio.close()
    
    return text

def pdfToText(pdfDir, absPath = os.getcwd(), txtDir = txtFilesDir):
    
    pdfPath = absPath+'/'+pdfDir
    txtPath = absPath+'/'+txtDir
    if pdfDir not in os.listdir(absPath):
        print('The specified directory "' + directory + '" does not exist')
        return
# Creates 'Text Files' directory for converted PDFs
    if txtDir not in os.listdir(absPath):
        os.mkdir(txtPath)
    
    docNum = 0
    stopAt = 25
    totalNum = len([file for file in os.scandir(pdfPath) if file.name.endswith('.pdf')])
    with open(absPath+'/'+'log.txt', 'a+', encoding="utf-8") as log:
        log.write("PDF to Text\n" + date.today().strftime("%m/%d/%y") +
                  " at " + datetime.now().strftime("%H:%M:%S") + "\n\n")        
# Moves on to next entity if the current entity is not a PDF
        for entity in os.scandir(pdfPath):
            if not entity.name.endswith('.pdf') or entity.name[0] in '1234567890':
                continue
            docNum += 1
            index = -4 # Remove '.pdf' from file name when creating '.txt' file
            fileName = entity.name[:index]+'.txt'
            print("Now on '"+entity.name+"'. . . ", end='')
            
# This block attempts to read the PDF file, extract text from each page,
#     and write the text to a text file with the same name
# Some documents are protected, corrupted, etc. and text cannot be extracted
# Exceptions are recorded in log.txt
# hasError remains true until each step in the try block is complete
            if fileName not in os.listdir(txtPath): 
                hasError = True
                try:
                    text = getText(pdfPath+'/'+entity.name)
                    txtFile = open(txtPath+'/'+fileName, 'w+', encoding="utf-8")
                    txtFile.write(text)
                    print("done\n")
                    hasError = False
#                     except TypeError as e:
#                         log.write("TypeError: " + entity.name + ":\n" + str(e))
#                     except PyPDF2.utils.PdfReadError as e:
#                         log.write("PdfReadError: " + entity.name + ":\n" + str(e))
#                     except OSError as e:
#                         log.write("OSError: " + entity.name + ":\n" + str(e))
#                     except decimal.InvalidOperation as e:
#                         log.write("InvalidOperation: " + entity.name + ":\n" + str(e))
                except Exception as e:
                    log.write(str(docNum)+": " + entity.name + ": \n\t" + str(e)+"\n")

                if hasError:
                    print("there was an error reading this document. See log for details. Reference number "+str(docNum)+".\n")
            else:
                max = 10
                if len(fileName) > max:
                    print('"'+fileName[:max]+'"' + '...', end='')
                else:
                    print('"'+fileName+'"', end='')
                print(' already exists\n')
            if docNum == stopAt:
                print("PDF to Text was stopped after "+str(docNum)+" documents.")
                break
        log.write("\n\n")
pdfToText('Working', absolute)

Now on 'Lafferty_pcfg-notes.pdf'. . . 

done

Now on 'LearningExecutableSemanticParsers.pdf'. . . done

Now on 'p123-zhang.pdf'. . . 

done

Now on 'Partial Parsing Finite-State Cascades.pdf'. . . done

Now on 'QueryEffectiveness.pdf'. . . 

done

Now on 'Text Clustering Algorithms.pdf'. . . 

done

Now on 'Text-Analytics-and-Natural-Language-Processing--.pdf'. . . done

Now on 'Workshop on Robust Methods in Analysis of Natural Language Data.pdf'. . . 

done



In [29]:
# Work on 10 (good) files at a time until pipeline works
#   then incrementally add files and clean up errors

# Function to remove \n
def rmvN(txtDir = txtFilesDir, rtnDir = rtnFilesDir, absPath = os.getcwd()):
    txtPath = absPath+'/'+txtDir
    if txtDir not in os.listdir(absPath):
        print('The specified directory "' + txtPath + '" does not exist')
        return
    rtnPath = absPath+'/'+rtnDir
    if rtnDir not in os.listdir(absPath):
        os.mkdir(rtnPath)
        
    for entity in os.scandir(txtPath):
        print("Now on '"+entity.name+"'. . . ", end='')
        with open(txtPath+'/'+entity.name, 'r+', encoding='utf-8') as txtFile:
            with open(rtnPath+'/'+entity.name, 'w+', encoding='utf-8') as rtnFile:
                text = txtFile.read()
                text = re.sub('-\n', '', text)
                text = re.sub('\n', '', text)
                rtnFile.write(text)
                rtnFile.truncate()
        print("done")
rmvN(absPath = absolute)

Now on 'Lafferty_pcfg-notes.txt'. . . done
Now on 'LearningExecutableSemanticParsers.txt'. . . done
Now on 'p123-zhang.txt'. . . done
Now on 'Partial Parsing Finite-State Cascades.txt'. . . done
Now on 'QueryEffectiveness.txt'. . . done
Now on 'Text Clustering Algorithms.txt'. . . done
Now on 'Text-Analytics-and-Natural-Language-Processing--.txt'. . . done
Now on 'Workshop on Robust Methods in Analysis of Natural Language Data.txt'. . . done


In [30]:
# Funtion to move files without spaces to new 'Without Spaces' directory         
def checkSpaces(txtDir = rtnFilesDir, spacesDir = spaceFilesDir, absPath = os.getcwd()):
    txtPath = absPath+'/'+txtDir
    if txtDir not in os.listdir(absPath):
        print('The specified directory "' + txtPath + '" does not exist')
        return
    spacesPath = absPath+'/'+spacesDir
    if spacesDir not in os.listdir(absPath):
        os.mkdir(spacesPath)
        
    with open(absPath+'/'+'Spaces.txt', 'a+', encoding='utf-8') as spaces: 
        spaces.write("Check Spaces\n" + date.today().strftime("%m/%d/%y") +
                  " at " + datetime.now().strftime("%H:%M:%S") + "\n\n")
        for entity in os.scandir(txtPath):
            print("Now on '"+entity.name+"'. . . ", end='')
            txtFile = open(txtPath+'/'+entity.name, 'r', encoding='utf-8')
            text = txtFile.read()
            split = text.split(' ')
            if len(split) < len(text)/10 or len(text) < 100 or text == '':
                txtFile.close()
                spaces.write(entity.name+'\n')
                if entity.name not in os.listdir(spacesPath):
                    os.rename(txtPath+'/'+entity.name, spacesPath+'/'+entity.name)
                else:
                    os.remove(txtPath+'/'+entity.name)
            print("done")
        spaces.write('\n\n')
checkSpaces(absPath = absolute) 

Now on 'Lafferty_pcfg-notes.txt'. . . done
Now on 'LearningExecutableSemanticParsers.txt'. . . done
Now on 'p123-zhang.txt'. . . done
Now on 'Partial Parsing Finite-State Cascades.txt'. . . done
Now on 'QueryEffectiveness.txt'. . . done
Now on 'Text Clustering Algorithms.txt'. . . done
Now on 'Text-Analytics-and-Natural-Language-Processing--.txt'. . . done
Now on 'Workshop on Robust Methods in Analysis of Natural Language Data.txt'. . . done


In [34]:
# Function to remove stopwords
# NLTK or SpaCy
# Inverted File: gram:[doc1, doc3] or gram:[[doc1,freq], [doc3,freq]]
def rmvStopWords(nlp, txtDir = rtnFilesDir, swDir = swFilesDir, absPath = os.getcwd()):
    txtPath = absPath+'/'+txtDir
    if txtDir not in os.listdir(absPath):
        print('The specified directory "' + txtPath + '" does not exist')
        return
    swPath = absPath+'/'+swDir
    if swDir not in os.listdir(absPath):
        os.mkdir(swPath)

    for entity in os.scandir(txtPath):
        print("Now on '"+entity.name+"'. . . ", end='')
        with open(txtPath+'/'+entity.name, 'r+', encoding='utf-8') as txtFile:
            with open(swPath+'/'+entity.name, 'w+', encoding='utf-8') as swFile:
                doc = nlp(txtFile.read())
                noStopWords = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct and not token.text.isnumeric()]
                swFile.write(" ".join(noStopWords))
                swFile.truncate()
        print("done")

rmvStopWords(nlp, absPath = absolute)

Now on 'Lafferty_pcfg-notes.txt'. . . done
Now on 'LearningExecutableSemanticParsers.txt'. . . done
Now on 'p123-zhang.txt'. . . done
Now on 'Partial Parsing Finite-State Cascades.txt'. . . done
Now on 'QueryEffectiveness.txt'. . . done
Now on 'Text Clustering Algorithms.txt'. . . done
Now on 'Text-Analytics-and-Natural-Language-Processing--.txt'. . . done
Now on 'Workshop on Robust Methods in Analysis of Natural Language Data.txt'. . . done


In [48]:
# Remove non-english words
def rmvNonEng(txtDir = swFilesDir, engDir = engFilesDir, absPath = os.getcwd()):
    txtPath = absPath+'/'+txtDir
    if txtDir not in os.listdir(absPath):
        print('The specified directory "' + txtPath + '" does not exist')
        return
    engPath = absPath+'/'+engDir
    if engDir not in os.listdir(absPath):
        os.mkdir(engPath)
    with open(absPath+'/'+'words_dictionary.json') as json_file:
        words = json.load(json_file)
        
    lets = []
    alph = 'abcdefghijklmnopqrstuvwxyz'
    for let in alph:
        lets.append(let)
        for char in alph:
            lets.append(let+char)
        
    for entity in os.scandir(txtPath):
        print("Now on '"+entity.name+"'. . . ", end='')
        with open(txtPath+'/'+entity.name, 'r+', encoding='utf-8') as txtFile:
            with open(engPath+'/'+entity.name, 'w+', encoding='utf-8') as engFile:
                text = txtFile.read().split(' ')
                engChars = [word for word in text if word in words and word not in lets]
                engFile.write(" ".join(engChars))
                engFile.truncate()
        print("done")
rmvNonEng(absPath = absolute)

Now on 'Lafferty_pcfg-notes.txt'. . . done
Now on 'LearningExecutableSemanticParsers.txt'. . . done
Now on 'p123-zhang.txt'. . . done
Now on 'Partial Parsing Finite-State Cascades.txt'. . . done
Now on 'QueryEffectiveness.txt'. . . done
Now on 'Text Clustering Algorithms.txt'. . . done
Now on 'Text-Analytics-and-Natural-Language-Processing--.txt'. . . done
Now on 'Workshop on Robust Methods in Analysis of Natural Language Data.txt'. . . done


In [3]:
# Stem words in all documents
def stem(txtDir = engFilesDir, stemDir = stemFilesDir, absPath = os.getcwd()):
    txtPath = absPath+'/'+txtDir
    if txtDir not in os.listdir(absPath):
        print('The specified directory "' + txtPath + '" does not exist')
        return
    stemPath = absPath+'/'+stemDir
    if stemDir not in os.listdir(absPath):
        os.mkdir(stemPath)
        
    stemmer = SnowballStemmer(language='english')
    for entity in os.scandir(txtPath):
        print("Now on '"+entity.name+"'. . . ", end='')
        with open(txtPath+'/'+entity.name, 'r+', encoding='utf-8') as txtFile:
            with open(stemPath+'/'+entity.name, 'w+', encoding='utf-8') as stemFile:
                text = txtFile.read().split(' ')
                stemmed = [stemmer.stem(word) for word in text]
                stemFile.write(" ".join(stemmed))
                stemFile.truncate()
        print("done")
stem(absPath = absolute)

Now on 'Lafferty_pcfg-notes.txt'. . . done
Now on 'LearningExecutableSemanticParsers.txt'. . . done
Now on 'p123-zhang.txt'. . . done
Now on 'Partial Parsing Finite-State Cascades.txt'. . . done
Now on 'QueryEffectiveness.txt'. . . done
Now on 'Text Clustering Algorithms.txt'. . . done
Now on 'Text-Analytics-and-Natural-Language-Processing--.txt'. . . done
Now on 'Workshop on Robust Methods in Analysis of Natural Language Data.txt'. . . done


In [35]:
# Create JSON with file_name : doc_id
def update_doc_index(jsonDoc = jsonDocIndex, txtDir = stemFilesDir, absPath = os.getcwd()):
    txtPath = absPath+'/'+txtDir
    jsonPath = absPath+'/'+jsonDoc
    if jsonDoc in os.listdir(absPath):
        with open(jsonPath, 'r') as jsonFile:
            docIndex = json.load(jsonFile)
    else:
        docIndex = {}
    for fileName in os.listdir(txtPath):
        if fileName not in docIndex.values():
            docIndex[len(docIndex)+1] = fileName
    with open(jsonPath, 'w') as jsonFile:
        json.dump(docIndex, jsonFile, indent=4)

update_doc_index(absPath = absolute)

In [37]:
# Create JSON with {doc_id1 : {"gram1":freq, "gram2":freq}, doc_id2 : {"gram1":freq}}
def update_inv_index(ngrams = 1, jsonDoc = jsonDocIndex, jsonInv = jsonInvIndex, txtDir = stemFilesDir, absPath = os.getcwd()):
    txtPath = absPath+'/'+txtDir
    jsonDocPath = absPath+'/'+jsonDoc
    if txtDir not in os.listdir(absPath):
        print('The specified directory "' + txtPath + '" does not exist')
        return
    if jsonDoc not in os.listdir(absPath):
        print('FILE NOT FOUND: The specified file "' + txtPath + '" does not exist')
        return
    jsonInvPath = absPath+'/'+jsonInv
    if jsonInv in os.listdir(absPath):
        with open(jsonInvPath, 'r') as jsonInv:
            invIndex = json.load(jsonInv)
    else:
        invIndex = {}
    with open(jsonDocPath, 'r') as jsonDoc:
        docIndex = json.load(jsonDoc)
    ngramStr = str(ngrams)+"-gram"
    if ngramStr not in invIndex:
        invIndex[ngramStr] = {}
    for docID in docIndex:
        if docID not in invIndex[ngramStr]:
            invIndex[ngramStr][docID] = {}
            with open(txtPath+'/'+docIndex[docID], 'r', encoding='utf-8') as txtFile:
                text = txtFile.read().split(' ')
                while len(text) > ngrams-1:
                    words = " ".join(text[:ngrams])
                    if words in invIndex[ngramStr][docID]:
                        invIndex[ngramStr][docID][words] += 1
                    else:
                        invIndex[ngramStr][docID][words] = 1
                    text.pop(0)
    with open(jsonInvPath, 'w') as jsonInv:
        json.dump(invIndex, jsonInv, indent=4)

update_inv_index(3, absPath = absolute)
                

In [51]:
import math
def calcCosSim(list1, list2):
    if len(list1) != len(list2):
        print("Lists do not have the same number of elements")
        return -1
    sumProd = sum(n1*n2 for n1, n2 in zip(list1, list2))
    sumL1 = sum(n**2 for n in list1)
    sumL2 = sum(n**2 for n in list2)
    
    return sumProd/(math.sqrt(sumL1)*math.sqrt(sumL2))

# Create vectors for all n-grams (with freq>1?) between two docs, multiply them,
#     then divide by the product Euclidian norms
# Print out similar phrases
def cossim(ngrams = 1, jsonDoc = jsonDocIndex, jsonInv = jsonInvIndex, absPath = os.getcwd()):
    if jsonDoc not in os.listdir(absPath):
        print('FILE NOT FOUND: The specified file "' + txtPath + '" does not exist')
        return
    jsonDocPath = absPath+'/'+jsonDoc
    if jsonInv not in os.listdir(absPath):
        print('FILE NOT FOUND: The specified file "' + txtPath + '" does not exist')
        return
    jsonInvPath = absPath+'/'+jsonInv
    ngramStr = str(ngrams) + "-gram"
    with open(jsonDocPath, 'r') as jsonDoc:
        docIndex = json.load(jsonDoc)
    with open(jsonInvPath, 'r') as jsonInv:
        invIndex = json.load(jsonInv)
    for doc1 in invIndex[ngramStr]:
        doc1Terms = []
        for term in invIndex[ngramStr][doc1]:
            doc1Terms.append(term)
        for doc2 in invIndex[ngramStr]:
            if doc2 > doc1:
                print(docIndex[doc1]+" and "+docIndex[doc2])
                allTerms = doc1Terms.copy()
                likeTerms = []
                for term in invIndex[ngramStr][doc2]:
                    if term not in allTerms:
                        allTerms.append(term)
                    else:
                        likeTerms.append(term)
                allTerms.sort()
                doc1freq = []
                doc2freq = []
                for term in allTerms:
                    doc1freq.append(invIndex[ngramStr][doc1].get(term, 0))
                    doc2freq.append(invIndex[ngramStr][doc2].get(term, 0))
                cosSimilarity = calcCosSim(doc1freq, doc2freq)
                print("Like terms (doc1, doc2):")
                for term in likeTerms:
                    print("\t"+term+" ("+str(invIndex[ngramStr][doc1][term])+", "+str(invIndex[ngramStr][doc2][term])+")")
                numLike = len(likeTerms)
                print("Doc1 has "+str(len(doc1Terms)-numLike)+" unique terms")
                print("Doc2 has "+str(len(allTerms)-len(doc1Terms))+" unique terms")
                print("Total like terms: \t"+str(numLike))
                print("Total unlike terms: \t"+str(len(allTerms)-numLike))
                print("Cosine Similarity: "+ str(cosSimilarity), end='\n\n')

cossim(3, absPath = absolute)
    

Lafferty_pcfg-notes.txt and LearningExecutableSemanticParsers.txt
Like terms (doc1, doc2):
	context free grammar (4, 1)
	maxim likelihood train (1, 1)
	natur languag grammar (1, 1)
Doc1 has 1092 unique terms
Doc2 has 2899 unique terms
Total like terms: 	3
Total unlike terms: 	3991
Cosine Similarity: 0.002663781873090507

Lafferty_pcfg-notes.txt and p123-zhang.txt
Like terms (doc1, doc2):
Doc1 has 1095 unique terms
Doc2 has 1739 unique terms
Total like terms: 	0
Total unlike terms: 	2834
Cosine Similarity: 0.0

Lafferty_pcfg-notes.txt and Partial Parsing Finite-State Cascades.txt
Like terms (doc1, doc2):
Doc1 has 1095 unique terms
Doc2 has 1202 unique terms
Total like terms: 	0
Total unlike terms: 	2297
Cosine Similarity: 0.0

Lafferty_pcfg-notes.txt and QueryEffectiveness.txt
Like terms (doc1, doc2):
Doc1 has 1095 unique terms
Doc2 has 1016 unique terms
Total like terms: 	0
Total unlike terms: 	2111
Cosine Similarity: 0.0

Lafferty_pcfg-notes.txt and Text Clustering Algorithms.txt
Like

Like terms (doc1, doc2):
Doc1 has 7567 unique terms
Doc2 has 2360 unique terms
Total like terms: 	0
Total unlike terms: 	9927
Cosine Similarity: 0.0

Text Clustering Algorithms.txt and Workshop on Robust Methods in Analysis of Natural Language Data.txt
Like terms (doc1, doc2):
	base text document (1, 5)
	chapter associ comput (1, 2)
	associ comput linguist (1, 8)
	annual meet associ (1, 1)
	confer european chapter (1, 1)
	european chapter associ (1, 1)
	model natur languag (1, 1)
	latent semant analysi (2, 4)
	inform storag retriev (1, 1)
	number give cluster (1, 1)
	section describ method (1, 1)
	document cluster cluster (4, 1)
	improv cluster qualiti (2, 2)
	frequenc invers document (1, 3)
	invers document frequenc (3, 2)
	document frequenc invers (1, 1)
	hierarch cluster method (1, 1)
	base cluster cluster (1, 1)
	cluster cluster document (1, 1)
	cluster averag similar (1, 1)
	averag similar pair (1, 1)
	similar pair document (1, 1)
	pair document cluster (1, 1)
	model inform retrie

In [5]:
# Calculate cosine similarity score for each document
# Files: id file_name; [id,]
def tfdif_cossim(txtDir = txtFilesDir, absPath = os.getcwd()):
    txtPath = absPath+'/'+filesDir
    if txtDir not in os.listdir(absPath):
        print('The specified directory "' + directory + '" does not exist')
        return
    vectorizer = TfidfVectorizer(ngram_range=(3,3), stop_words=None, use_idf=True)
    X2 = vectorizer.fit_transform([open(directory+'/'+file, "r", encoding='utf-8').read() for file in os.listdir(directory)[:25]])
#     X2 = vectorizer.fit_transform(open(directory+'/'+'A Bible for the Disability Field.txt', 'r', encoding='utf-8'))
    features = vectorizer.get_feature_names()
    print(features)
    scores = (X2.toarray()) 
#     print("\n\nScores : \n", scores) 

    # Getting top ranking features 
    sums = X2.sum(axis = 0) 
    data1 = [] 
    for col, term in enumerate(features): 
        data1.append( (term, sums[0, col] )) 
    ranking = pd.DataFrame(data1, columns = ['term', 'rank']) 
    words = (ranking.sort_values('rank', ascending = False)) 
    print ("\n\nWords : \n", words.head(50)) 
    
# tfdif_cossim(txtFilesDir, absolute)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)





Words : 
                              term      rank
114277           scrum scrum meet  0.495291
17558           arxiv 1412 6980v9  0.408248
7453                 6980v9 cs lg  0.408248
5566                  30 jan 2017  0.408248
75278                   lg 30 jan  0.408248
36343                    cs lg 30  0.408248
1149               1412 6980v9 cs  0.408248
38145        decis support system  0.389146
104414             quarter vol 25  0.321385
48376        eni organiz communic  0.321385
139423                vol 25 june  0.316201
12142                af mil af073  0.302430
10990         adapt decis support  0.299287
45049           dual use commerci  0.297304
136891      use commerci militari  0.297304
28678    commerci militari applic  0.297304
81569   mental retard development  0.246098
111366  retard development disabl  0.246098
93855         peopl mental retard  0.218242
45888           educ train mental  0.211735
88217        normal truncat model  0.208413
115191        self d

In [ ]:
# Trigram parser-based inverted file 
# (TF-DIF to remove trigrams common to most or all documents)

In [ ]:
# Clustering algorithm based on trigram inverted file

In [ ]:
# Add bigram parser-based info to inverted file

In [ ]:
# Implement clustering on bigram inverted file